In [2]:
from gs_quant.target.instrument import EqOption
from gs_quant.risk import EqAnnualImpliedVol, Price, EqSpot, EqDelta
from gs_quant.session import GsSession, Environment
from gs_quant.common import MeasureScenario
from gs_quant.markets import PricingContext
import datetime as dt

In [4]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.QA, client_id=None, client_secret=None, scopes=('run_analytics',))

MeasureScenario class allows us to override inputs such as spot or volatility when pricing an instrument. Here are some examples of how to use it with FX Options.

In [ ]:
# Initial FX Option
eq_option = EqOption('.SPX')
init_eq_option = eq_option.calc((EqSpot, EqDelta, Price))
init_eq_option.to_frame()

We can override inputs such as spot or volatility when pricing an instrument. Here are some examples of how to use it with Equities Options. We might be also interested in comparing delta calculated by the system and the approximation derived from the price change. 

In [ ]:
# Override a single input (Spot) with MeasureScenario context manager
with MeasureScenario(EqSpot, init_eq_option[EqSpot] * 1.01):
    eq_risks_up = eq_option.calc((EqSpot, EqDelta, Price))
eq_risks_up.to_frame()

In [ ]:
with MeasureScenario(EqSpot, init_eq_option[EqSpot] * 0.99):
    eq_risks_down = eq_option.calc((EqSpot, EqDelta, Price))
eq_risks_down.to_frame()

In [ ]:
print(init_eq_option[EqDelta] / 100)
print(((eq_risks_up[Price] - eq_risks_down[Price]) / 2))

We can see that the delta values were close, which is expected.

We can override multiple inputs this way.

In [ ]:
# Override multiple inputs (Spot rate and Annual Implied Volatility) with MeasureScenario context manager and return multiple outputs
with MeasureScenario(EqSpot, 6000):
    with MeasureScenario(EqAnnualImpliedVol, 0.2):
        eq_risks = eq_option.calc((Price, EqAnnualImpliedVol, EqSpot))
eq_risks.to_frame()

We can also combine the measure override with other context managers, such as PricingContext

In [ ]:
# Set up time context for pricing, and override the input
with PricingContext(dt.date(2025, 7, 5)):
    with MeasureScenario(EqAnnualImpliedVol, 0.2):
        eq_risks = eq_option.calc((Price, EqAnnualImpliedVol, EqSpot))
eq_risks.result().to_frame()